In [3]:
import sys
import time
from opt import *
from metrics import accuracy, auc, prf, metrics
from dataload import dataloader
from model import fc_hgnn
import os
from dataload import LabelSmoothingLoss
from dataload import Logger

# Load preset initial parameters.
opt = OptInit().initialize()

# Create log text
filename = opt.log_path
log = Logger(filename)
sys.stdout = log

# Create a data loader.
dl = dataloader()
raw_features, y, nonimg, phonetic_score, time_series, dynamic_fc = dl.load_data()

# Initializes the data partition for ten-fold cross-validation.
n_folds = opt.n_folds
cv_splits = dl.data_split(n_folds)

# Create evaluation metrics
corrects = np.zeros(n_folds, dtype=np.int32) 
accs = np.zeros(n_folds, dtype=np.float32) 
sens = np.zeros(n_folds, dtype=np.float32) 
spes = np.zeros(n_folds, dtype=np.float32) 
aucs = np.zeros(n_folds, dtype=np.float32)
prfs = np.zeros([n_folds,3], dtype=np.float32)

 Using GPU in torch
==========       CONFIG      =============
train:1
use_cpu:False
lr:0.01
wd:5e-05
num_iter:400
dropout:0.3
num_classes:2
n_folds:5
ckpt_path:./
log_path:./inffus_log.txt
subject_IDs_path:/mnt/test_data/ABIDE_pcp/subject_IDs.txt
phenotype_path:/mnt/test_data/ABIDE_pcp/Phenotypic_V1_0b_preprocessed1.csv
data_path:/mnt/test_data/ABIDE_pcp/cpac/filt_noglobal
alpha:0.65
beta:1.5
k1:0.9
k2:0.5
time:250409
device:cuda:0
==========     CONFIG END    =============


===> Phase is train.


In [38]:
cv_splits[0]

(array([  0,   2,   3,   5,   6,   7,   8,   9,  10,  11,  12,  13,  14,
         15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  28,
         29,  30,  31,  32,  33,  35,  37,  38,  40,  41,  42,  44,  45,
         47,  48,  49,  51,  52,  53,  54,  55,  56,  57,  59,  60,  62,
         63,  64,  65,  66,  68,  69,  70,  71,  73,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  88,  90,  91,  92,  93,  94,
         95,  98,  99, 100, 101, 102, 103, 104, 106, 108, 109, 111, 112,
        114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        129, 130, 131, 132, 133, 134, 135, 136, 137, 139, 140, 141, 142,
        143, 145, 146, 147, 150, 151, 152, 153, 155, 156, 158, 159, 160,
        161, 162, 163, 164, 166, 167, 171, 172, 173, 174, 175, 176, 178,
        179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 191, 192,
        194, 195, 199, 200, 202, 203, 204, 205, 206, 207, 208, 209, 210,
        211, 213, 214, 215, 216, 217, 218, 219, 220

In [40]:
labels = torch.tensor(y, dtype=torch.long).to(opt.device)
labels

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [31]:
embeddings = []
for graph in raw_features:
    embedding= graph.x.reshape(1,-1)
    embeddings.append(embedding)
embeddings = torch.cat(tuple(embeddings))
embeddings.shape

torch.Size([871, 12321])

In [ ]:
# 将 torch 张量转换为 numpy 数组
embeddings_np = embeddings.numpy()

# 使用 t-SNE 将特征映射到二维空间
tsne = TSNE(n_components=2, perplexity=55, learning_rate=5, n_iter=5000)
tsne_embeddings = tsne.fit_transform(embeddings_np)

# 定义颜色映射
colors = {0: 'red', 1: 'blue'}
label_colors = [colors[int(label.cpu())] for label in labels]

# 可视化 t-SNE 结果
plt.scatter(tsne_embeddings[:, 0], tsne_embeddings[:, 1], c=label_colors)
plt.title('t-SNE Visualization of Original Embeddings')
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')

# 添加图例
legend_elements = [
    plt.Line2D([0], [0], marker='o', color='w', label='ASD', markerfacecolor='red', markersize=10),
    plt.Line2D([0], [0], marker='o', color='w', label='HC', markerfacecolor='blue', markersize=10)
]
plt.legend(handles=legend_elements)

plt.show()

In [2]:
# The training set and test set in this fold.
train_ind = cv_splits[0][0]
test_ind = cv_splits[0][1]

# The labels and model in this fold.
labels = torch.tensor(y, dtype=torch.long).to(opt.device)
model = fc_hgnn(nonimg, phonetic_score).to(opt.device)
print(model)

# The loss function in this fold.
# We found that the use of the label smoothing function provided a modest performance boost.
# The cross-entropy function is used in this paper.
loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn =LabelSmoothingLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=opt.lr, weight_decay=opt.wd)
fold_model_path = opt.ckpt_path + r"\inffus_fold{}.pth".format(fold)

# The train function.
def train():
    acc = 0
    for epoch in range(opt.num_iter):
        # Model training in this epoch.
        model.train()
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            node_logits = model(raw_features, time_series, dynamic_fc)
            loss_cla = loss_fn(node_logits[train_ind], labels[train_ind])
            loss = loss_cla
            loss.backward()
            optimizer.step()
        correct_train, acc_train = accuracy(node_logits[train_ind].detach().cpu().numpy(), y[train_ind])

        # Model evaluating in this epoch.
        model.eval()
        with torch.set_grad_enabled(False):
            node_logits= model(raw_features, time_series, dynamic_fc)
        logits_test = node_logits[test_ind].detach().cpu().numpy()
        correct_test, acc_test = accuracy(logits_test, y[test_ind])
        test_sen, test_spe = metrics(logits_test, y[test_ind])
        auc_test = auc(logits_test,y[test_ind])
        prf_test = prf(logits_test,y[test_ind])

        # Output the training and test results of this epoch.
        print("Epoch: {},\tce loss: {:.5f},\tce loss_cla: {:.5f},\ttrain acc: {:.5f},\ttest acc: {:.5f},\ttest spe: {:.5f},\ttest sen: {:.5f}     ".format(epoch, loss.item(),loss_cla.item(),acc_train.item(),acc_test.item(),test_spe,test_sen),time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time())))

        # Save the best results and parameters so far.
        if acc_test > acc:
            acc = acc_test
            correct = correct_test
            aucs[fold] = auc_test
            prfs[fold] = prf_test
            sens[fold] = test_sen
            spes[fold] = test_spe
            if opt.ckpt_path != '':
                if not os.path.exists(opt.ckpt_path):
                    os.makedirs(opt.ckpt_path)
                torch.save(model.state_dict(), fold_model_path)
                print("{} Saved model to:{}".format("\u2714", fold_model_path))

    # Output the test results of this fold.
    accs[fold] = acc
    corrects[fold] = correct
    print("\r\n => Fold {} test accuacry {:.5f}".format(fold, acc))


# The evaluate function.
def evaluate():
    print("  Number of testing samples %d" % len(test_ind))
    print('  Start testing...')
    model.load_state_dict(torch.load(fold_model_path))
    model.eval()
    node_logits = model(raw_features, time_series, dynamic_fc)
    logits_test = node_logits[test_ind].detach().cpu().numpy()
    corrects[fold], accs[fold] = accuracy(logits_test, y[test_ind])
    sens[fold], spes[fold] = metrics(logits_test, y[test_ind])
    aucs[fold] = auc(logits_test, y[test_ind])
    prfs[fold] = prf(logits_test, y[test_ind])
    print("  Fold {} test accuracy {:.5f}, AUC {:.5f},".format(fold, accs[fold], aucs[fold]))

# Select the model for training or evaluation.
train()

In [ ]:
import pandas as pd
import numpy as np
from sklearn.manifold import TSNE
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# 假设这是你的模型类
class FC_HGNN:
    def __init__(self):
        # 初始化模型参数
        pass

    def forward(self, x):
        # 模型前向传播
        return x


# 加载数据，这里假设数据已经被预处理成合适的形式
# 示例数据，实际使用时请替换为真实数据
data = pd.read_csv('your_data.csv')
X = data.drop('label', axis=1).values
y = data['label'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 将数据转换为torch张量
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# 初始化模型
model = FC_HGNN()

# 获取学习前的特征
X_train_before = X_train
X_test_before = X_test

# 通过模型获取学习后的特征
with torch.no_grad():
    X_train_after = model.forward(X_train_tensor).numpy()
    X_test_after = model.forward(X_test_tensor).numpy()

# 使用t-SNE进行特征映射
tsne = TSNE(n_components=2, random_state=42)
X_train_before_tsne = tsne.fit_transform(X_train_before)
X_test_before_tsne = tsne.transform(X_test_before)
X_train_after_tsne = tsne.fit_transform(X_train_after)
X_test_after_tsne = tsne.transform(X_test_after)


# 绘制学习前的样本嵌入表示
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(X_train_before_tsne[:, 0], X_train_before_tsne[:, 1], c=y_train, cmap='viridis')
plt.title('Before learning - Training data')
plt.xlabel('t-SNE dimension 1')
plt.ylabel('t-SNE dimension 2')

plt.subplot(1, 2, 2)
plt.scatter(X_test_before_tsne[:, 0], X_test_before_tsne[:, 1], c=y_test, cmap='viridis')
plt.title('Before learning - Test data')
plt.xlabel('t-SNE dimension 1')
plt.ylabel('t-SNE dimension 2')

plt.show()


# 绘制学习后的样本嵌入表示
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.scatter(X_train_after_tsne[:, 0], X_train_after_tsne[:, 1], c=y_train, cmap='viridis')
plt.title('After learning - Training data')
plt.xlabel('t-SNE dimension 1')
plt.ylabel('t-SNE dimension 2')

plt.subplot(1, 2, 2)
plt.scatter(X_test_after_tsne[:, 0], X_test_after_tsne[:, 1], c=y_test, cmap='viridis')
plt.title('After learning - Test data')
plt.xlabel('t-SNE dimension 1')
plt.ylabel('t-SNE dimension 2')

plt.show()